Importing the required libraries

In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import pickle


Loading the required x_train, y_train, x_test, y_test

In [3]:
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv").squeeze()  # Convert to Series
X_test = pd.read_csv("data/X_test.csv")
y_test = pd.read_csv("data/y_test.csv").squeeze()

Building the Perceptron

In [7]:
# Convert DataFrame to NumPy array before converting to tensor
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [8]:
# Create DataLoaders
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Training the Perceptron

In [9]:
# Define the SLP model
class SingleLayerPerceptron(nn.Module):
    def __init__(self, input_size):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1)  # Single layer
        self.sigmoid = nn.Sigmoid()  # Activation function

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
input_size = X_train.shape[1]  # Number of features
model = SingleLayerPerceptron(input_size)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        y_batch = y_batch.float().unsqueeze(1)  # Reshape for BCELoss
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/20, Loss: 0.5938
Epoch 2/20, Loss: 0.5097
Epoch 3/20, Loss: 0.4797
Epoch 4/20, Loss: 0.4640
Epoch 5/20, Loss: 0.4548
Epoch 6/20, Loss: 0.4481
Epoch 7/20, Loss: 0.4432
Epoch 8/20, Loss: 0.4402
Epoch 9/20, Loss: 0.4391
Epoch 10/20, Loss: 0.4362
Epoch 11/20, Loss: 0.4356
Epoch 12/20, Loss: 0.4343
Epoch 13/20, Loss: 0.4337
Epoch 14/20, Loss: 0.4337
Epoch 15/20, Loss: 0.4338
Epoch 16/20, Loss: 0.4326
Epoch 17/20, Loss: 0.4319
Epoch 18/20, Loss: 0.4307
Epoch 19/20, Loss: 0.4322
Epoch 20/20, Loss: 0.4315


In [11]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = (outputs > 0.5).float()  # Convert probabilities to 0 or 1
        correct += (predicted.squeeze() == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.7927


In [ ]:
Building a mlp model

In [14]:
# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        
        layers = []
        prev_size = input_size

        # Create hidden layers
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())  # Activation function
            prev_size = hidden_size

        # Output layer
        layers.append(nn.Linear(prev_size, output_size))
        layers.append(nn.Sigmoid())  # Sigmoid for binary classification

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [15]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).long()
X_test_tensor = torch.from_numpy(X_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).long()

# Define Hyperparameters
hidden_layer_configs = [[32], [64, 32], [128, 64, 32]]  # Different layer combinations
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64]
num_epochs = 30
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold Cross Validation

In [16]:
def train_and_evaluate(hidden_sizes, learning_rate, batch_size):
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = MLP(input_size=X_train_tensor.shape[1], hidden_sizes=hidden_sizes, output_size=1)
    criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y.float())
            loss.backward()
            optimizer.step()

    # Evaluation
    with torch.no_grad():
        test_outputs = model(X_test_tensor).squeeze()
        test_predictions = (test_outputs >= 0.5).long()
        accuracy = (test_predictions == y_test_tensor).float().mean().item()

    return accuracy

In [17]:
best_config = None
best_accuracy = 0

for hidden_sizes in hidden_layer_configs:
    for lr in learning_rates:
        for batch_size in batch_sizes:
            accuracies = []
            for train_idx, val_idx in kf.split(X_train_tensor):
                X_train_fold, X_val_fold = X_train_tensor[train_idx], X_train_tensor[val_idx]
                y_train_fold, y_val_fold = y_train_tensor[train_idx], y_train_tensor[val_idx]

                acc = train_and_evaluate(hidden_sizes, lr, batch_size)
                accuracies.append(acc)

            mean_acc = np.mean(accuracies)
            print(f"Config: {hidden_sizes}, LR: {lr}, Batch: {batch_size}, Acc: {mean_acc:.4f}")

            if mean_acc > best_accuracy:
                best_accuracy = mean_acc
                best_config = (hidden_sizes, lr, batch_size)

print(f"Best Config: {best_config} with Accuracy: {best_accuracy:.4f}")

Config: [32], LR: 0.01, Batch: 16, Acc: 0.7883
Config: [32], LR: 0.01, Batch: 32, Acc: 0.7891
Config: [32], LR: 0.01, Batch: 64, Acc: 0.7887
Config: [32], LR: 0.001, Batch: 16, Acc: 0.7923
Config: [32], LR: 0.001, Batch: 32, Acc: 0.7913
Config: [32], LR: 0.001, Batch: 64, Acc: 0.7911
Config: [32], LR: 0.0001, Batch: 16, Acc: 0.7892
Config: [32], LR: 0.0001, Batch: 32, Acc: 0.7885
Config: [32], LR: 0.0001, Batch: 64, Acc: 0.7780
Config: [64, 32], LR: 0.01, Batch: 16, Acc: 0.7885
Config: [64, 32], LR: 0.01, Batch: 32, Acc: 0.7874
Config: [64, 32], LR: 0.01, Batch: 64, Acc: 0.7873
Config: [64, 32], LR: 0.001, Batch: 16, Acc: 0.7859
Config: [64, 32], LR: 0.001, Batch: 32, Acc: 0.7883
Config: [64, 32], LR: 0.001, Batch: 64, Acc: 0.7879
Config: [64, 32], LR: 0.0001, Batch: 16, Acc: 0.7913
Config: [64, 32], LR: 0.0001, Batch: 32, Acc: 0.7911
Config: [64, 32], LR: 0.0001, Batch: 64, Acc: 0.7927
Config: [128, 64, 32], LR: 0.01, Batch: 16, Acc: 0.7867
Config: [128, 64, 32], LR: 0.01, Batch: 32, 

In [20]:
# Evaluating the best model
best_hidden_sizes, best_lr, best_batch_size = best_config
best_accuracy = train_and_evaluate(best_hidden_sizes, best_lr, best_batch_size)
print(f"Best Accuracy: {best_accuracy:.4f}")


Best Accuracy: 0.7943


In [24]:
#confusion matrix and classification report
y_pred = model(X_test_tensor).squeeze()
y_pred = (y_pred >= 0.5).long()
confusion = confusion_matrix(y_test_tensor, y_pred)
print("Confusion Matrix:")
print(confusion)
classification_report = classification_report(y_test_tensor, y_pred)
print("Classification Report:")
print(classification_report)

Confusion Matrix:
[[1421  332]
 [ 410  837]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.81      0.79      1753
           1       0.72      0.67      0.69      1247

    accuracy                           0.75      3000
   macro avg       0.75      0.74      0.74      3000
weighted avg       0.75      0.75      0.75      3000



The best model has:
Hidden Layers: [64, 32]
Learning Rate: 0.0001
Batch Size: 64
Accuracy: 0.7927 (79.27%)

A two-layer MLP (64, 32 neurons) worked better than simpler or deeper models.
A lower learning rate (0.0001) gave better stability and accuracy.
A batch size of 64 was optimal for efficient training.

Saving the best model

In [26]:
# Saving the best model
torch.save(model.state_dict(), 'best_model.pth')
print("Model saved successfully!")

Model saved successfully!
